In [1]:
import pandas as pd
import os

The viewing activity is too big, we need to filter it only for nobel prize winners:

In [26]:
WINNERS_PATH = '../../data/novel-prize/raw/nobel_laureates_data.csv'
WIKIPEDIA_2015_PATH = '../../data/novel-prize/raw/wikipedia/monthly_views_2015.csv'
WIKIPEDIA_2016_PATH = '../../data/novel-prize/raw/wikipedia/monthly_views_2016.csv'
WIKIPEDIA_2017_PATH = '../../data/novel-prize/raw/wikipedia/monthly_views_2017.csv'
WIKIPEDIA_2018_PATH = '../../data/novel-prize/raw/wikipedia/monthly_views_2018.csv'
WIKIPEDIA_2019_PATH = '../../data/novel-prize/raw/wikipedia/monthly_views_2019.csv'
WIKIPEDIA_2020_PATH = '../../data/novel-prize/raw/wikipedia/monthly_views_2020.csv'
WIKIPEDIA_2021_PATH = '../../data/novel-prize/raw/wikipedia/monthly_views_2021.csv'
WIKIPEDIA_2022_PATH = '../../data/novel-prize/raw/wikipedia/monthly_views_2022.csv'
WIKIPEDIA_2023_PATH = '../../data/novel-prize/raw/wikipedia/monthly_views_2023.csv'

In [2]:
winners_df = pd.read_csv(WINNERS_PATH)
wikipedia_2015_df = pd.read_csv(WIKIPEDIA_2015_PATH)

In [3]:
wikipedia_2015_df.head()

,title,month,monthly_views
0,Anthony_van_Hoboken,2015-08-01 00:00:00 UTC,256
1,Brian_Conklin,2015-08-01 00:00:00 UTC,256
2,Julian_Seward,2015-08-01 00:00:00 UTC,256
3,Greg_Ducre,2015-08-01 00:00:00 UTC,256
4,Richard_H._Ellis,2015-08-01 00:00:00 UTC,256


In [4]:
winners_df.head()

,year,category,motivation,prizeShare,laureateID,fullName,gender,born,bornCountry,bornCity,died,diedCountry,diedCity,organizationName,organizationCountry,organizationCity
0,2023,medicine,for their discoveries concerning nucleoside ba...,2,1024,Katalin Kariko,female,17-01-1955,Hungary,Szolnok,0000-00-00,NaN,NaN,Szeged University,Hungary,Szeged
1,2023,economics,for having advanced our understanding of women...,1,1034,Claudia Goldin,female,1946-00-00,USA,New York NY,0000-00-00,NaN,NaN,Harvard University,USA,Cambridge MA
2,2023,peace,for her fight against the oppression of women ...,1,1033,Narges Mohammadi,female,21-04-1972,Iran,Zanjan,0000-00-00,NaN,NaN,NaN,NaN,NaN
3,2023,literature,for his innovative plays and prose which give ...,1,1032,Jon Fosse,male,29-09-1959,Norway,Haugesund,0000-00-00,NaN,NaN,NaN,NaN,NaN
4,2023,chemistry,for the discovery and synthesis of quantum dots,3,1031,Alexei Ekimov,male,1945-00-00,Russia,NaN,0000-00-00,NaN,NaN,Nanocrystals Technology Inc.,USA,New York NY


### Steps:
1. Match their titles to the same format
2. group by date instead of mounth
3. filter by novel prize winners and year
4. do it for all data sets and get new and matching verions
5. connect all wikipidia data sets into one
6. remove unmatched rows (match means both the winner name and the year identical)
7. remove redundant columns from the winners df
8. connect and get new data set
9. name data transformatinos to enable leraning


#### 1. Match their titles to the same format

In [5]:
def match_titles(wikipedia_df):
    updated_wikipedia_df = wikipedia_df.copy()
    #remove from meomery
    wikipedia_df = pd.DataFrame()
    updated_wikipedia_df['title'] = updated_wikipedia_df['title'].str.replace('_', ' ', regex=False)
    return updated_wikipedia_df

In [6]:
wikipedia_2015_df = match_titles(wikipedia_df=wikipedia_2015_df)

In [19]:
wikipedia_2015_df.head()

,title,year,yearly_views
0,!PAUS3,2015,1025
1,"""Bassy"" Bob Brockmann",2015,1099
2,"""Big"" Donnie MacLeod",2015,466
3,"""By"" Dunham",2015,34
4,"""Country"" Johnny Mathis",2015,60


### 2. Filter By Novel Prize Winners:

#### 2. group by date instead of mounth

In [14]:
def convert_to_year_measurment(wikipedia_df):
    updated_wikipedia_df = wikipedia_df.copy()
    #remove from meomery
    wikipedia_df = pd.DataFrame()
    # Convert 'mounth' to datetime
    updated_wikipedia_df['month'] = pd.to_datetime(updated_wikipedia_df['month'])
    # Extract the year from 'mounth'
    updated_wikipedia_df['year'] = updated_wikipedia_df['month'].dt.year
    # Group by 'title' and 'year', then sum the 'activity'
    updated_wikipedia_df = updated_wikipedia_df.groupby(['title', 'year'], as_index=False)['monthly_views'].sum()
    updated_wikipedia_df = updated_wikipedia_df.rename(columns={'monthly_views':'yearly_views'})

    return updated_wikipedia_df

In [15]:
wikipedia_2015_df = convert_to_year_measurment(wikipedia_df=wikipedia_2015_df)

### 3. Filter By Novel Prize Winners:

In [20]:
def filter_by_novel_prize_winers(wikipedia_df, winners_df):
    filtered_wikipedia_df = wikipedia_df[wikipedia_df[['title', 'year']].apply(
    tuple, axis=1).isin(winners_df[['fullName', 'year']].apply(tuple, axis=1))]
    return filtered_wikipedia_df

In [21]:
wikipedia_2015_df = filter_by_novel_prize_winers(wikipedia_df=wikipedia_2015_df, winners_df=winners_df)

#### 4. do it for all data sets and get new and matching verions

first let's some it all for one method:

In [23]:
def get_updated_wikipedia_yearly_df(wikipedia_df, winners_df):
    wikipedia_df = match_titles(wikipedia_df=wikipedia_df)
    wikipedia_df = convert_to_year_measurment(wikipedia_df=wikipedia_df)
    wikipedia_df = filter_by_novel_prize_winers(wikipedia_df=wikipedia_df, winners_df=winners_df)
    return wikipedia_df

In [24]:
def get_all_updated_wikipedia_dfs(wikipedia_paths, winners_df):
    dfs = []
    for wikipedia_path in wikipedia_paths:
        wikipedia_df = pd.read_csv(wikipedia_path)
        updated_wikipdia_df = get_updated_wikipedia_yearly_df(wikipedia_df=wikipedia_df, winners_df=winners_df)
        dfs.append(updated_wikipdia_df)
    return dfs

In [27]:
wikipedia_paths = [WIKIPEDIA_2015_PATH, WIKIPEDIA_2016_PATH, WIKIPEDIA_2017_PATH, WIKIPEDIA_2018_PATH, WIKIPEDIA_2019_PATH, WIKIPEDIA_2020_PATH, WIKIPEDIA_2021_PATH, WIKIPEDIA_2022_PATH, WIKIPEDIA_2023_PATH]
wikipedia_dfs = get_all_updated_wikipedia_dfs(wikipedia_paths=wikipedia_paths, winners_df=winners_df)

#### 5. connect all wikipidia data sets into one

In [28]:
def connect_wikipedia_dfs(dfs):
    full_wikipedia_df = pd.concat(dfs, ignore_index=True)
    return full_wikipedia_df

In [29]:
full_wikipedia_df = connect_wikipedia_dfs(dfs=wikipedia_dfs)

#### 6. Match columns names

In [35]:
def match_columns_names(full_wikipedia_df, winners_df):
    full_wikipedia_df = full_wikipedia_df.rename(columns={'title':'name'})
    winners_df = winners_df.rename(columns={'fullName':'name'})
    return full_wikipedia_df, winners_df

In [36]:
full_wikipedia_df, winners_df = match_columns_names(full_wikipedia_df=full_wikipedia_df, winners_df=winners_df)

#### 7. Remove unmatched rows (match means both the winner name and the year identical)

In [38]:
def remove_unmatched_rows(full_wikipedia_df, winners_df):
    matched_wikipedia_df = pd.merge(full_wikipedia_df, winners_df, how='inner', on=['name', 'year'])
    matching_wikipedia = full_wikipedia_df[full_wikipedia_df[['name', 'year']].apply(tuple, axis=1).isin(matched_wikipedia_df[['name', 'year']].apply(tuple, axis=1))]
    matching_winners = winners_df[winners_df[['name', 'year']].apply(tuple, axis=1).isin(matched_wikipedia_df[['name', 'year']].apply(tuple, axis=1))]
    return matching_wikipedia, matching_winners

In [39]:
full_wikipedia_df, winners_df = remove_unmatched_rows(full_wikipedia_df=full_wikipedia_df, winners_df=winners_df)

In [43]:
winners_df.head()

,year,category,motivation,prizeShare,laureateID,name,gender,born,bornCountry,bornCity,died,diedCountry,diedCity,organizationName,organizationCountry,organizationCity
1,2023,economics,for having advanced our understanding of women...,1,1034,Claudia Goldin,female,1946-00-00,USA,New York NY,0000-00-00,NaN,NaN,Harvard University,USA,Cambridge MA
2,2023,peace,for her fight against the oppression of women ...,1,1033,Narges Mohammadi,female,21-04-1972,Iran,Zanjan,0000-00-00,NaN,NaN,NaN,NaN,NaN
3,2023,literature,for his innovative plays and prose which give ...,1,1032,Jon Fosse,male,29-09-1959,Norway,Haugesund,0000-00-00,NaN,NaN,NaN,NaN,NaN
6,2023,chemistry,for the discovery and synthesis of quantum dots,3,1029,Moungi Bawendi,male,1961-00-00,France,Paris,0000-00-00,NaN,NaN,Massachusetts Institute of Technology (MIT),USA,Cambridge MA
8,2023,physics,for experimental methods that generate attosec...,3,1027,Ferenc Krausz,male,17-05-1962,Hungary,Mor,0000-00-00,NaN,NaN,Max Planck Institute of Quantum Optics,Germany,Garching


#### 8. Remove duplicated winners:

In [44]:
def remove_duplicated_winners(winners_df):
    winners_df_unique = winners_df.drop_duplicates(subset=['name', 'year'], keep='first')
    return winners_df_unique


In [45]:
winners_df = remove_duplicated_winners(winners_df=winners_df)

#### 9. remove redundant columns from the winners df


In [48]:
def remove_redundant_columns(winners_df):
    return winners_df.drop(columns=['motivation','laureateID', 'bornCity', 'died', 'diedCountry', 'diedCity','organizationCity'])

In [49]:
winners_df = remove_redundant_columns(winners_df=winners_df)

#### 9. connect and get new data set

In [56]:
def coombine_data_sets(full_wikipedia_df, winners_df):
    combined_df = pd.merge(full_wikipedia_df, winners_df, on=['name', 'year'], how='right')
    return combined_df


In [57]:
combined_df = coombine_data_sets(full_wikipedia_df=full_wikipedia_df, winners_df=winners_df)

#### 10. df transformations:

In [70]:
def transform_data(full_wikipedia_df, winners_df, combined_df):
    l_full_wikipedia_df = full_wikipedia_df.copy()
    l_winners_df = winners_df.copy()
    l_combined_df = combined_df.copy()


    l_winners_df = pd.get_dummies(l_winners_df, columns=['gender', 'bornCountry', 'organizationCountry'])
    l_combined_df = pd.get_dummies(combined_df, columns=['gender', 'bornCountry', 'organizationCountry'])

    l_winners_df['category'], _ = pd.factorize(l_winners_df['category'])
    l_combined_df['category'], _ = pd.factorize(l_combined_df['category'])

    l_winners_df['organizationName'], _ = pd.factorize(l_winners_df['organizationName'])
    l_combined_df['organizationName'], _ = pd.factorize(l_combined_df['organizationName'])

    l_winners_df['name'], _ = pd.factorize(l_winners_df['name'])
    l_combined_df['name'], _ = pd.factorize(l_combined_df['name'])
    l_full_wikipedia_df['name'], _ = pd.factorize(l_full_wikipedia_df['name'])



    return l_full_wikipedia_df, l_winners_df, l_combined_df

In [71]:
l_full_wikipedia_df, l_winners_df, l_combined_df = transform_data(full_wikipedia_df=full_wikipedia_df, winners_df=winners_df, combined_df=combined_df)

#### Write back as csv

In [67]:
def write_dfs_as_csv(dfs):
    for filename, df in dfs.items():
        file_path = os.path.join('../../data/novel-prize/processed', f"{filename}.csv")
        df.to_csv(file_path, index=False)

In [72]:
write_dfs_as_csv(dfs=
{
    'eda_full_wikipedia':full_wikipedia_df,
    'learning_full_wikipedia':l_full_wikipedia_df,
    'eda_winners':winners_df,
    'learning_winners':l_winners_df,
    'eda_combined_novel_prize':combined_df,
    'learning_combined_novel_prize':l_combined_df,
})